In [6]:
import numpy as np
from scipy import interpolate
from scipy.io import wavfile

import matplotlib.pyplot as plt
# from matplotlib import cm as colormaps

from importlib import reload

from topography import bathymetry as Bathy
import environment as Env
import ray as Ray
import simulation as Sim

In [7]:
reload(Bathy)

map_path = 'src/asc/MNT_MED100m_GDL-CA_HOMONIM_WGS84_NM_ZNEG.asc'
map = Bathy.Map.from_asc(map_path)

In [8]:
start = (2.8995, 41.6995)  # lower left
stop = (7.9005, 41.6995)  # lower right

x, z = map.cut(start, stop, npoints=100)
floor = interpolate.interp1d(x, z)

ceiling = lambda x: 0 * np.ones_like(x)

In [9]:
floor = lambda x: -5000 * np.ones_like(x)

In [10]:
reload(Ray)
reload(Env)
reload(Sim)

source = np.array([0., 0.])
target = np.array([3500, -1000])

env = Env.Environment2D (floor, ceiling)

sim = Sim.EigenraySim2D (
    env, source, target,
    dx_max=50, dz_max=25, backprop=False, n_rebounds_max=6,
    n_rays_scan=1000, n_rays=25, scan_angle_min=-1.5, scan_angle_max=1.5,
    verbose=True, verbose_depth_max=0
)

sim.refine(n_rays=20, cone_half_angle=0.01)  # Initial refine (empirical: cone_half_angle = 10/n_rays_scan)
sim.refine(iterations=5)

EigenraySim2D: Scanning using 1000 rays between angles -1.5 and 1.5


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/scipy/optimize/minpack.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


EigenraySim2D: Scan mean distance: 29.569971562793043


ValueError: A value in x_new is above the interpolation range.

In [ ]:
### EIGENSIM TESTS ###

figsize = (100, 30)

# fig = plt.figure(figsize=figsize)
# sim.plot(fig, pack='__iter0-scan')
# plt.scatter((source[0], target[0]), (source[1], target[1]), figure=fig, c='red', s=1000)
# plt.title(f'Scanning rays')
# plt.show(fig)

fig = plt.figure(figsize=figsize)
sim.plot(fig, pack=sim.pack_scan, c='black')
plt.scatter((source[0], target[0]), (source[1], target[1]), figure=fig, c='red', s=1000)
plt.title(f'Scan output')
plt.show(fig)

for rid in range(1, sim.n_refines+1):
    fig = plt.figure(figsize=figsize)
    sim.plot(fig, pack=sim.gen_pack_refine(rid), c='black')
    plt.scatter((source[0], target[0]), (source[1], target[1]), figure=fig, c='red', s=1000)
    plt.title(f'Refine {rid}')
    plt.show(fig)

In [67]:
path_in = 'src/wav/sig-test.wav'
sample_rate, signal = wavfile.read(path_in)
signal = signal[:, 0]

In [ ]:
freqs = np.linspace(0, 24000, 100)
filter = sim.get_filter (*freqs)

In [68]:
sample_rate, signal_filtered = filter(sample_rate, signal)
signal_filtered = signal_filtered.astype(signal.dtype)

path_out = 'out/sig-test-filtered.wav'
wavfile.write(path_out, sample_rate, signal_filtered.astype(np.int16))